In [ ]:
from pettingzoo.utils.env import ParallelEnv

In [ ]:
import networkx as nx
import functools

import gymnasium
import numpy as np
from gymnasium.spaces import Discrete

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers
import matplotlib.pyplot as plt

## One thief and one Police environment

In [ ]:
class CustomEnvironment(ParallelEnv):
    
    metadata = {
        "name": "custom_graph_environment_v1",
    }
    def __init__(self):
        # this is the graph
        self.g_env = nx.read_graphml('g1.gml')
        self.g_no_node = len(self.g_env.nodes())
        self.node_list = list(self.g_env.nodes())
        
        # A dictionary that relates each discrete value in the observation
        # space to a node obtained from the graph
        
            # 1. Create empty dictionary
        self.node_dict = {}
        self.node_inv_dict = {}
        
            # 2. relating the key to the value of the node
        for key,value in enumerate(self.node_list):
            self.node_dict[key] = value 
            
            # 3. relating the value to the key ( used later on )
        self.node_inv_dict = {value: key for key, value in self.node_dict.items()}
        
        # sets the maximum steps after which the program will terminate 
        self.max_steps = 100
        self.step_now = 0
        
        
        self.possible_agents = ["player_" + str(r) for r in range(2)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )
        # {'player_0': 0, 'player_1': 1}

        
        self._action_spaces = {agent: Discrete(4) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Discrete(self.g_no_node**len(self.possible_agents)) for agent in self.possible_agents
        }
        
        self.agents = self.possible_agents
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: None for agent in self.agents}
        
        # not utilized
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        
                
        # this is extra things for visualization you do not need to know
        graph = self.g_env
        position = list(graph.nodes())
        position = [self.str_to_tuple(name) for name in position]
        pos = dict(zip(graph.nodes(), position))
        self.node_positions = pos
        
        # sets the current state of agents
        # sets the thief to position 6
        self.state['player_0'] = self.node_dict[6]
        
        # sets the police to position 9
        self.state['player_1'] = self.node_dict[9]
        
        
    def reset(self, seed=None, options=None):
        self.timestep = None
        self.state = {agent: None for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.step_now = 0
        
        # sets the thief to position 6
        self.state['player_0'] = self.node_dict[6]
        
        # sets the police to position 9
        self.state['player_1'] = self.node_dict[9]
        return self.state

    def step(self, actions):
        
        # observation returns the next state of the agents
        # for each action selected for the agent the observations sould be sent back
        # the impletentation is bad change if possible
        terminations = False
        rewards = {"player_0":None, "player_1":None}
        # taking the actions (input) 
        thief_action = actions["player_0"]
        police_action = actions["player_1"]
        
        # current states
        thief_state = self.state['player_0']
        police_state = self.state['player_1']
        
        # possible_thief are the possible places the thief can go 
        possible_thief = [i for i in self.g_env.neighbors(thief_state)]
        possible_police = [i for i in self.g_env.neighbors(police_state)]
        
        
        # movement of the thief and the police according to the action and their rewards       
        for agent in self.possible_agents:
            temp_neighbours = []
            for neighbour in self.g_env.neighbors(self.state[agent]):
                temp_neighbours.append(neighbour)
    
            if actions[agent] < len([i for i in self.g_env.neighbors(self.state[agent])]):
                rewards[agent] = -2;
                self.state[agent] = temp_neighbours[actions[agent]]
                self.step_now += 1

            elif actions[agent] == len([i for i in self.g_env.neighbors(self.state[agent])]):
                rewards[agent] = -1;
                self.step_now += 1
            else:
                rewards[agent] = -10;
                
            # checking for terminations
            # nice one
#             for index in possible_thief:
#                 for i in range(len(possible_police)):
#                     if(index == possible_police[i]):
#                         terminations = True;    
            if(police_state == thief_state):
                terminations = True
            for index in possible_thief:
                if(index == police_state):
                    terminations = True;
        
        # extra reward for thief for running away
        if (self.step_now>7):
                rewards['player_0'] += 20
                rewards['player_1'] -= 20
        # Get dummy infos (not used in this example)
        infos = {a: {} for a in self.agents}
        observations = {a: self.state[a] for a in self.agents}
        truncations = {a: None for a in self.agents}
        
        return observations, rewards, terminations, truncations, infos
    
    def render(self):
        pass
    
    # only works for 2 players
    def temp_render(self,episode):
        
        nx.draw(self.g_env, self.node_positions,node_size=300)
        nx.draw_networkx_labels(self.g_env, self.node_positions,labels = self.node_inv_dict,font_color='black' )
        
        x1,y1 = self.node_positions[str(self.state['player_0'])]
        x2,y2 = self.node_positions[str(self.state['player_1'])]
        
        filename = f"images/Multi_{episode}_{self.step_now}.png"
        
        plt.scatter(x2, y2, s=550, c='purple')
        plt.scatter(x1, y1, s=450, c='red')    
        
        plt.savefig(filename)
        plt.show()

    def observation_space(self, agent):
        return self.observation_spaces[agent]

    def action_space(self, agent):
        return self.action_spaces[agent]
    
    def str_to_tuple(self,string):
        return tuple(float(x) for x in string.strip('()').split(','))
    
    def possible_move_range(self,player):
        return len(list(self.g_env.neighbors(self.state[player])))

In [ ]:
env = CustomEnvironment()
episodes = 1

for episode in range(1,episodes+1):
    state = env.reset()
    terminations = False
    score = {a:0 for a in env.possible_agents}
    print(state)
    while not terminations:
        n0 = np.random.randint(0,env.possible_move_range('player_0')+1)
        n1 = np.random.randint(0,env.possible_move_range('player_1')+1)
        print(n0)
        print(n1)
        env.temp_render(episode)
        action = {'player_0':n0,'player_1':n1}
        observations, rewards, terminations, truncations, infos = env.step(action)
        if (terminations == True):
            print('terminating episode = {}'.format(env.step_now))
        for agent in env.possible_agents:
            score[agent] += rewards[agent]
    print('Episode: {} Score: {} \n'.format(episode,score))

## Example of implementation of rock paper scissor game 

In [ ]:
ROCK = 0
PAPER = 1
SCISSORS = 2
NONE = 3
MOVES = ["ROCK", "PAPER", "SCISSORS", "None"]
NUM_ITERS = 100
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}


def env(render_mode=None):
    """
    The env function often wraps the environment in wrappers by default.
    You can find full documentation for these methods
    elsewhere in the developer documentation.
    """
    internal_render_mode = render_mode if render_mode != "ansi" else "human"
    env = raw_env(render_mode=internal_render_mode)
    # This wrapper is only for environments which print results to the terminal
    if render_mode == "ansi":
        env = wrappers.CaptureStdoutWrapper(env)
    # this wrapper helps error handling for discrete action spaces
    env = wrappers.AssertOutOfBoundsWrapper(env)
    # Provides a wide vareity of helpful user errors
    # Strongly recommended
    env = wrappers.OrderEnforcingWrapper(env)
    return env


class raw_env(AECEnv):
    """
    The metadata holds environment constants. From gymnasium, we inherit the "render_modes",
    metadata which specifies which modes can be put into the render() method.
    At least human mode should be supported.
    The "name" metadata allows the environment to be pretty printed.
    """

    metadata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        """
        The init method takes in environment arguments and
         should define the following attributes:
        - possible_agents
        - action_spaces
        - observation_spaces
        These attributes should not be changed after initialization.
        """
        self.possible_agents = ["player_" + str(r) for r in range(2)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )

        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Discrete(4) for agen t in self.possible_agents
        }
        self.render_mode = render_mode

    # this cache ensures that same space object is returned for the same agent
    # allows action space seeding to work as expected
    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        # gymnasium spaces are defined and documented here: https://gymnasium.farama.org/api/spaces/
        return Discrete(4)

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        """
        Renders the environment. In human mode, it can print to terminal, open
        up a graphical window, or open up some other display that a human can see and understand.
        """
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return

        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)

    def observe(self, agent):
        """
        Observe should return the observation of the specified agent. This function
        should return a sane observation (though not necessarily the most up to date possible)
        at any time after reset() is called.
        """
        # observation of one agent is the previous state of the other
        return np.array(self.observations[agent])

    def close(self):
        """
        Close should release any graphical displays, subprocesses, network connections
        or any other environment data which should not be kept around after the
        user is no longer using the environment.
        """
        pass

    def reset(self, seed=None, options=None):
        """
        Reset needs to initialize the following attributes
        - agents
        - rewards
        - _cumulative_rewards
        - terminations
        - truncations
        - infos
        - agent_selection
        And must set up the environment so that render(), step(), and observe()
        can be called without issues.
        Here it sets up the state dictionary which is used by step() and the observations dictionary which is used by step() and observe()
        """
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: NONE for agent in self.agents}
        self.observations = {agent: NONE for agent in self.agents}
        self.num_moves = 0
        """
        Our agent_selector utility allows easy cyclic stepping through the agents list.
        """
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    def step(self, action):
        """
        step(action) takes in an action for the current agent (specified by
        agent_selection) and needs to update
        - rewards
        - _cumulative_rewards (accumulating the rewards)
        - terminations
        - truncations
        - infos
        - agent_selection (to the next agent)
        And any internal state used by observe() or render()
        """
        if (
            self.terminations[self.agent_selection]
            or self.truncations[self.agent_selection]
        ):
            # handles stepping an agent which is already dead
            # accepts a None action for the one agent, and moves the agent_selection to
            # the next dead agent,  or if there are no more dead agents, to the next live agent
            self._was_dead_step(action)
            return

        agent = self.agent_selection

        # the agent which stepped last had its _cumulative_rewards accounted for
        # (because it was returned by last()), so the _cumulative_rewards for this
        # agent should start again at 0
        self._cumulative_rewards[agent] = 0

        # stores action of current agent
        self.state[self.agent_selection] = action

        # collect reward if it is the last agent to act
        if self._agent_selector.is_last():
            # rewards for all agents are placed in the .rewards dictionary
            self.rewards[self.agents[0]], self.rewards[self.agents[1]] = REWARD_MAP[
                (self.state[self.agents[0]], self.state[self.agents[1]])
            ]

            self.num_moves += 1
            # The truncations dictionary must be updated for all players.
            self.truncations = {
                agent: self.num_moves >= NUM_ITERS for agent in self.agents
            }

            # observe the current state
            for i in self.agents:
                self.observations[i] = self.state[
                    self.agents[1 - self.agent_name_mapping[i]]
                ]
        else:
            # necessary so that observe() returns a reasonable observation at all times.
            self.state[self.agents[1 - self.agent_name_mapping[agent]]] = NONE
            # no rewards are allocated until both players give an action
            self._clear_rewards()

        # selects the next agent.
        self.agent_selection = self._agent_selector.next()
        # Adds .rewards to ._cumulative_rewards
        self._accumulate_rewards()

        if self.render_mode == "human":
            self.render()

In [ ]:
# # reference prisoner guard environment

# import functools
# import random
# from copy import copy

# import numpy as np
# from gymnasium.spaces import Discrete, MultiDiscrete

# from pettingzoo.utils.env import ParallelEnv

# class CustomEnvironment(ParallelEnv):
#     metadata = {
#         "name": "custom_environment_v0",
#     }

#     def __init__(self):
#         self.escape_y = None
#         self.escape_x = None
#         self.guard_y = None
#         self.guard_x = None
#         self.prisoner_y = None
#         self.prisoner_x = None
#         self.timestep = None
#         self.possible_agents = ["prisoner", "guard"]

#     def reset(self, seed=None, options=None):
#         self.agents = copy(self.possible_agents)
#         self.timestep = 0

#         self.prisoner_x = 0
#         self.prisoner_y = 0

#         self.guard_x = 7
#         self.guard_y = 7

#         self.escape_x = random.randint(2, 5)
#         self.escape_y = random.randint(2, 5)

#         observations = {
#             a: (
#                 self.prisoner_x + 7 * self.prisoner_y,
#                 self.guard_x + 7 * self.guard_y,
#                 self.escape_x + 7 * self.escape_y,
#             )
#             for a in self.agents
#         }
#         return observations

#     def step(self, actions):
#         # Execute actions
#         prisoner_action = actions["prisoner"]
#         guard_action = actions["guard"]

#         if prisoner_action == 0 and self.prisoner_x > 0:
#             self.prisoner_x -= 1
#         elif prisoner_action == 1 and self.prisoner_x < 6:
#             self.prisoner_x += 1
#         elif prisoner_action == 2 and self.prisoner_y > 0:
#             self.prisoner_y -= 1
#         elif prisoner_action == 3 and self.prisoner_y < 6:
#             self.prisoner_y += 1

#         if guard_action == 0 and self.guard_x > 0:
#             self.guard_x -= 1
#         elif guard_action == 1 and self.guard_x < 6:
#             self.guard_x += 1
#         elif guard_action == 2 and self.guard_y > 0:
#             self.guard_y -= 1
#         elif guard_action == 3 and self.guard_y < 6:
#             self.guard_y += 1

#         # Check termination conditions
#         terminations = {a: False for a in self.agents}
#         rewards = {a: 0 for a in self.agents}
#         if self.prisoner_x == self.guard_x and self.prisoner_y == self.guard_y:
#             rewards = {"prisoner": -1, "guard": 1}
#             terminations = {a: True for a in self.agents}

#         elif self.prisoner_x == self.escape_x and self.prisoner_y == self.escape_y:
#             rewards = {"prisoner": 1, "guard": -1}
#             terminations = {a: True for a in self.agents}

#         # Check truncation conditions (overwrites termination conditions)
#         truncations = {a: False for a in self.agents}
#         if self.timestep > 100:
#             rewards = {"prisoner": 0, "guard": 0}
#             truncations = {"prisoner": True, "guard": True}
#             self.agents = []
#         self.timestep += 1

#         # Get observations
#         observations = {
#             a: (
#                 self.prisoner_x + 7 * self.prisoner_y,
#                 self.guard_x + 7 * self.guard_y,
#                 self.escape_x + 7 * self.escape_y,
#             )
#             for a in self.agents
#         }

#         # Get dummy infos (not used in this example)
#         infos = {a: {} for a in self.agents}

#         return observations, rewards, terminations, truncations, infos

#     def render(self):
#         grid = np.zeros((7, 7))
#         grid[self.prisoner_y, self.prisoner_x] = "P"
#         grid[self.guard_y, self.guard_x] = "G"
#         grid[self.escape_y, self.escape_x] = "E"
#         print(f"{grid} \n")

#     @functools.lru_cache(maxsize=None)
#     def observation_space(self, agent):
#         return MultiDiscrete([7 * 7 - 1] * 3)

#     @functools.lru_cache(maxsize=None)
#     def action_space(self, agent):
#         return Discrete(4)